# Crashes Near Point

Find the crash incident_id for a specific crash given its location and time. 

In [1]:
import folium
import datetime
import pandas as pd
from folium.plugins import HeatMap

import os
os.chdir('..')
from scripts.crash_data_analysis import CrashDataAnalysis

In [2]:
cda = CrashDataAnalysis()
# df = cda.crash_dataframe(verbose=True)

In [3]:
# geocoded_location = 'S Zuni St and W Arkansas Ave, Denver, CO'
# geocoded_location = 'Godsman Elementary School, Denver, CO'
# geocoded_location = 'W 26th Ave and Tennyson St, Denver, CO'
# geocoded_location = 'E 1st Ave and N Gilpin St, Denver, CO'

# crash occurred within this distance of the given latitude and longitude
radius_miles = 0.5

In [4]:
# lat_lon = cda.geocode_location(geocoded_location)
# lat_lon

In [12]:
geocoded_location = 'E 1st Ave and N Gilpin St, Denver, CO'
lat_lon = [39.71824953362511, -104.96668328236497] # 'E 1st Ave and N Gilpin St, Denver, CO'

In [6]:
query = f"""
select
*
, reported_date at time zone 'America/Denver' as reported_date_local
, ST_Distance(
    geom_denver, ST_Transform(ST_SetSRID(ST_MakePoint({lat_lon[1]},{lat_lon[0]}), 4326), 3502)
) as distance_feet

from crashes

where geo_lon is not null
and ST_Distance(
    geom_denver, ST_Transform(ST_SetSRID(ST_MakePoint({lat_lon[1]},{lat_lon[0]}), 4326), 3502)
    ) < {radius_miles * 5280}

order by reported_date
"""
# and extract(year from reported_date) >= 2022

nearby = pd.read_sql(query, cda.conn)

nearby['reported_date_local'] = nearby['reported_date_local'].dt.tz_localize('America/Denver')

In [7]:
nearby.groupby(nearby.reported_date.dt.year).agg(
    crashes=('incident_id', 'count')
    , fatalities=('fatality', 'sum')
    , sbi=('sbi', 'sum')
    , pedestrian_ind=('pedestrian_ind', 'sum')
    , bicycle_ind=('bicycle_ind', 'sum')
).astype(int) #.to_clipboard()

,crashes,fatalities,sbi,pedestrian_ind,bicycle_ind
reported_date,,,,,
2013,141,0,0,4,2
2014,156,1,1,2,2
2015,159,2,1,4,2
2016,156,2,2,4,4
2017,139,0,0,4,2
2018,144,0,2,2,5
2019,154,1,0,0,4
2020,103,0,2,0,2
2021,99,0,3,1,0


In [8]:
nearby['all_time'] = 'all time'
nearby.groupby('all_time').agg(
    crashes=('incident_id', 'count')
    , fatalities=('fatality', 'sum')
    , sbi=('sbi', 'sum')
    , pedestrian_ind=('pedestrian_ind', 'sum')
    , bicycle_ind=('bicycle_ind', 'sum')
).astype(int)

,crashes,fatalities,sbi,pedestrian_ind,bicycle_ind
all_time,,,,,
all time,1612,6,19,25,33


In [9]:
display_df = nearby.copy()
display_df['bicycle'] = display_df['bicycle_ind'].replace(0, '').replace(1, True)
display_df['pedestrian'] = display_df['pedestrian_ind'].replace(0, '').replace(1, True)
display_df['serious_injury'] = display_df['sbi'].replace(False, '')
display_df['fatal'] = display_df['fatality'].replace(False, '')

display_cols = [
    'reported_date_local'
    , 'top_traffic_accident_offense'
    , 'incident_address_corrected'
    , 'bicycle'
    , 'pedestrian'
    , 'serious_injury'
    , 'fatal'
    , 'distance_feet'
    , 'incident_id'
]

# nearby.to_clipboard()
# nearby[display_cols]
# display_df.loc[display_df.pedestrian_ind == 1, display_cols]
display_df[display_cols].tail(10)

,reported_date_local,top_traffic_accident_offense,incident_address_corrected,bicycle,pedestrian,serious_injury,fatal,distance_feet,incident_id
1602,2025-02-20 18:17:00-07:00,TRAF - ACCIDENT,2400 E 2ND AVE,,,,,2469.970134,202588431
1603,2025-02-21 11:28:00-07:00,TRAF - ACCIDENT,200 BLOCK N JOSEPHINE ST,,,,,2487.720904,202589704
1604,2025-02-28 13:33:00-07:00,TRAF - ACCIDENT,2300 BLOCK E 1ST AVE,,,,,2571.426716,202510339
1605,2025-03-10 11:16:00-06:00,TRAF - ACCIDENT,1200 BLOCK E 4TH AVE,,,,,2132.670883,2025121503
1606,2025-03-11 16:59:00-06:00,TRAF - ACCIDENT - HIT & RUN,E 1ST AVE / N LAFAYETTE ST,,,,,1098.123347,2025124199
1607,2025-03-13 14:39:00-06:00,TRAF - ACCIDENT - HIT & RUN,N UNIVERSITY BLVD / E 1ST AVE,,,,,2098.994383,2025127949
1608,2025-03-16 14:38:00-06:00,TRAF - ACCIDENT,303 N JOSEPHINE ST,,,,,2526.731019,2025133509
1609,2025-03-20 12:41:00-06:00,TRAF - ACCIDENT - HIT & RUN,N DOWNING ST / E 1ST AVE,,,,,1842.366117,2025140787
1610,2025-03-21 04:07:00-06:00,TRAF - ACCIDENT - HIT & RUN,200 BLOCK S HUMBOLDT ST,,,,,2429.951861,2025142059
1611,2025-03-23 20:30:00-06:00,TRAF - ACCIDENT - HIT & RUN,E SPEER BLVD / N DOWNING ST,,,,,2034.206363,2025147040


In [10]:
this_map = folium.Map(prefer_canvas=True)
HeatMap(nearby[['geo_lat', 'geo_lon']]).add_to(this_map)
folium.Circle(location=lat_lon, weight=2, radius=radius_miles*1609.34).add_to(this_map)
this_map.fit_bounds(
    [nearby[['geo_lat', 'geo_lon']].min().values.tolist()
     , nearby[['geo_lat', 'geo_lon']].max().values.tolist()]
)
this_map

In [13]:
# Text string with frequency of crashes
num_years = 3

most_recent_timestamp = cda.most_recent_crash_timestamp()
start_timestamp = most_recent_timestamp - pd.DateOffset(years=num_years)
days_in_time_range = (most_recent_timestamp - start_timestamp).days

nearby_in_time_range = nearby[nearby.reported_date_local > start_timestamp].copy()

days_between_crashes = days_in_time_range / len(nearby_in_time_range)

print(
    f'The location "{geocoded_location}" has had {len(nearby_in_time_range)} traffic crashes'
    , f'within a {radius_miles} mile radius'
    , f'over the last {num_years} years, an average of {days_between_crashes:.1f} days between crashes.'
    , f'In that time, there were {nearby_in_time_range.fatality.sum()} deadly crashes'
    , f'and {nearby_in_time_range.sbi.sum()} crashes causing a serious bodily injury.'
    , f'There were {(nearby_in_time_range.pedestrian_ind > 0).sum()} crashes involving a pedestrian'
    , f'and {(nearby_in_time_range.bicycle_ind > 0).sum()} crashes involving a bicyclist.'
)

The location "E 1st Ave and N Gilpin St, Denver, CO" has had 340 traffic crashes within a 0.5 mile radius over the last 3 years, an average of 3.2 days between crashes. In that time, there were 0 deadly crashes and 8 crashes causing a serious bodily injury. There were 4 crashes involving a pedestrian and 10 crashes involving a bicyclist.


In [14]:
cda.conn.dispose()